### Use natural language to query database with financial transactions

#### Purpose:
Use natural language to filter the database and extract data that is then displayed on the app screen in a dashboard

In [ ]:
import pandas as pd
import sqlite3
import os
import openai
from datetime import datetime
import json
from langchain.prompts import PromptTemplate
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import time
import concurrent.futures
import warnings
import ast

In [ ]:
# Set the OpenAI API key. The API key is provided in the email and has a 5$ credit to start with.

os.environ['OPENAI_API_KEY'] = "--------- INSERT YOUR OPENAI API KEY HERE ---------"

In [ ]:
# set variables and paths

working_directory = os.getcwd()
images_path = os.path.join(working_directory, 'images')
llm_model = "gpt-4-turbo"
db_name = "Database_transactions.db"
client = openai.OpenAI()

# Get today's date in YYYY-MM-DD format
today_date = datetime(2025, 3, 4).strftime("%Y-%m-%d")
day_of_week = datetime(2025, 3, 4).strftime("%A")
preprocessing_time_threshold = 4  # a threshold for preprocessing time in seconds

### Utils

In [ ]:
def load_image(image_name, dpi = 300, scale = 1.5):
    # Load an image in notebook

    image_path = os.path.join(images_path, image_name)
    
    img = mpimg.imread(image_path)
    height, width = img.shape[:2]  # Extract height & width

    fig_width = width / dpi  # Calculate figure width in inches
    fig_height = height / dpi  # Calculate figure height in inches

    plt.figure(figsize=(fig_width * scale, fig_height* scale), dpi=dpi)  

    # Display the image
    plt.imshow(img)
    plt.axis("off")  # Hide axes
    plt.show()

# Timing Decorator
def timing_decorator(func):
    """Decorator to measure execution time of a function."""
    
    def wrapper(*args, **kwargs):
        start_time = time.time()  # Start timer
        result = func(*args, **kwargs)  # Execute the function
        end_time = time.time()  # End timer
        execution_time = end_time - start_time
        #print(f"✅ {func.__name__} executed in {execution_time:.4f} seconds")
        return result, execution_time
    return wrapper

# validation function
def validation(df_testing_results, benchmark_file = "benchmarks.csv"):
    
    # Load the results from the CSV file
    benchmarks = pd.read_csv(benchmark_file)

    benchmarks["Response"] = benchmarks["Response"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else None)
    comparison_df = df_testing_results.merge(benchmarks[['Original Query', 'Response']], left_on="Original Query", right_on="Original Query", suffixes=('_test', '_benchmark'))

    comparison_df['PASSED TEST?'] = comparison_df['Response_test'].eq(comparison_df['Response_benchmark'])
    return comparison_df

# Architecture

A SQLite database is created with two tables:
1) Transactions Table: the user's transactions (it is assumed that the transactions table is already filtered for a single user)
2) Merchants Table: the spending category linked to each Merchant (a 1-1 relation is assumed)

An example is shown below:

In [ ]:
load_image('test_set.png', dpi = 300, scale = 1)

### The design the process

- a user asks a question about financial transactions
- the query is processed, checking for spelling mistakes and maps activities to a finite set of categories (es food shopping -> Groceries)
- relevant features are exracted from the preprocessed query 
- the extracted features populate a predefined SQL query
- a structured database is called using the generated SQL query
- The results are returned

The process is shown in the image below:

In [ ]:
load_image('process_design.png', dpi = 300, scale = 1.5)

### Step 1. Create a SQLite database with a transactions table and a merchant table

In [ ]:
def create_db_sqlite(db_name=db_name):
    """
    Creates a SQLite database with a transactions table.
    If the table already exists, it is dropped and recreated with fresh data.
    """

    # Connect to SQLite database (creates the file if it doesn't exist)
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    # Drop the table if it already exists
    cursor.execute("DROP TABLE IF EXISTS transactions")
    cursor.execute("DROP TABLE IF EXISTS def_merchants")

    # Define the SQL query to create a new table
    create_transactions_table_query = """
    CREATE TABLE transactions (
        transaction_id TEXT PRIMARY KEY,
        date TEXT NOT NULL,
        amount REAL NOT NULL,
        merchant_id TEXT NOT NULL
    );
    """

    create_def_table_query = """
    CREATE TABLE def_merchants (
        merchant_id TEXT PRIMARY KEY,
        merchant TEXT NOT NULL,
        spending_cat TEXT NOT NULL
    );
    """

    # Execute the query to create the table
    cursor.execute(create_transactions_table_query)

    cursor.execute(create_def_table_query)

    # Define transaction data
    transactions = [
        ('T-00', '2023-06-15', 35.0, "M-01"),
        ('T-01', '2023-07-20', 50.0, "M-04"),
        ('T-02', '2023-09-05', 75.0, "M-05"),
        ('T-03', '2024-08-01', 43, "M-07"),
        ('T-04', '2024-08-01', 45.0, "M-03"),
        ('T-05', '2024-08-31', 49, "M-02"),
        ('T-06', '2024-08-31', 50.0, "M-03"),
        ('T-07', '2024-09-30', 55, "M-01"),
        ('T-08', '2024-09-30', 55.0, "M-04"),
        ('T-09', '2024-10-30', 61, "M-01"),
        ('T-10', '2024-10-30', 60.0, "M-03"),
        ('T-11', '2024-11-29', 67, "M-01"),
        ('T-12', '2024-11-29', 65.0, "M-03"),
        ('T-13', '2024-12-29', 73, "M-02"),
        ('T-14', '2024-12-29', 70.0, "M-03"),
        ('T-15', '2025-01-28', 79, "M-01"),
        ('T-16', '2025-01-28', 75.0, "M-06"),
        ('T-17', '2025-02-27', 85, "M-07"),
        ('T-18', '2025-02-27', 80.0, "M-03"),
        ('T-19', '2025-03-01', 20.0, "M-04"),
        ('T-20', '2025-03-01', 10.0, "M-01"),
        ('T-21', '2025-03-01', 40.0, "M-06"),
        ('T-22', '2025-03-02', 10.0, "M-01"),
        ('T-23', '2025-03-03', 60.0, "M-05"),
        ('T-24', '2025-03-03', 100.0, "M-04"),
        ('T-25', '2025-03-04', 40.0, "M-08"),
        ('T-26', '2025-03-04', 10.0, "M-06")]

    def_merchants = [("M-01", "TESCO", "GROCERIES"), 
                 ("M-02", "WAITROSE", "GROCERIES"), 
                 ("M-03", "VUE", "ENTERTAINMENT"), 
                 ("M-04", "STARLING", "FINANCE"), 
                 ("M-05", "SKY", "BILLS"), 
                 ("M-06", "MCDONALD'S", "EATING OUT"),
                 ("M-07", "STARBUCKS", "COFFEE"),
                  ("M-08", "MONZO", "FINANCE")]

    # Insert transaction data into the table
    cursor.executemany("INSERT INTO transactions VALUES (?, ?, ?, ?)", transactions)

    cursor.executemany("INSERT INTO def_merchants VALUES (?, ?, ?)", def_merchants)

    # Commit changes and close the connection
    conn.commit()

    print(f"Database '{db_name}' created successfully with 'transactions' table rewritten.")


    query = "SELECT transaction_id, date, amount, spending_cat, merchant FROM transactions LEFT JOIN def_merchants ON transactions.merchant_id = def_merchants.merchant_id ;"

    # Execute the query
    cursor.execute(query)

    # Fetch all results
    transactions = cursor.fetchall()
    first_transaction_date = transactions[0][1]
    # Print results
    print("\n\n\nDisplaying table: \n")
    #print("Merchant ID | Merchant       |Spending Category")  
    print("Transaction ID | Date       | Amount | Spending Category | Merchant")  
    print("-" * 70)
    for transaction in transactions:
        #print(f"{transaction[0]:<11} | {transaction[1]:<14} | {transaction[2]:<6}")
        print(f"{transaction[0]:<14} | {transaction[1]:<10} | {transaction[2]:<6} | {transaction[3]:<18}| {transaction[4]:<10}")

    # Close the connection
    conn.close()
    return first_transaction_date

first_transaction_date = create_db_sqlite()

### Step 2. Define the functions and mappings 

In [ ]:
# Predefined spending categories and merchants for normalization
CATEGORY_MAPPING = {
    "GROCERIES": ["food shop", "supermarket", "shopping"],
    "ENTERTAINMENT": ["movies", "cinema", "fun", "leisure"],
    "EATING OUT": ["restaurant", "dining", "takeaway"],
    "COFFEE": ["cafe", "starbucks", "coffee shop"],
    "FINANCE": ["bank", "investment"],
    "BILLS": ["utilities", "electricity", "internet"]
}

function_calling = [
    {
        "name": "extract_query_features",
        "description": "Extracts key features from a user query related to transactions, including date ranges (formatted as YYYY-MM-DD), merchant names, spending categories, and the required SQL aggregation function.",
        "parameters": {
            "type": "object",
            "properties": {
                "start_date": {
                    "type": "string",
                    "format": "date",
                    "description": f"""The start date extracted from the query (YYYY-MM-DD). If the query specifies a relative time frame (e.g., 'last week', 'last 4 months'), compute the actual date based on today's date  (Today is {today_date}, {day_of_week}).If only a month is mentioned, assume the first day of that month and the current year from today date.
                                    If the query does not specify a start date, use the earliest date available in the database, namely {first_transaction_date}. the week starts on Monday and ends on Sunday. If the date format is ambiguous, assume it follows the DD-MM-YYYY format"""
                },
                "end_date": {
                    "type": "string",
                    "format": "date",
                    "description": f"""The end date extracted from the query (YYYY-MM-DD). If the query specifies a relative time frame, compute the actual date based on today's date (Today is {today_date}, {day_of_week}). If only a month is mentioned, assume the last day of that month and the current year from today date.
                    If the date format is ambiguous, assume it follows the DD-MM-YYYY format."""
                },
                "merchant": {
                    "type": "string",
                    "enum": ["TESCO", "WAITROSE", "VUE", "STARLING", "SKY", "MCDONALD'S", "STARBUCKS", "MONZO"],
                    "nullable": True,
                    "description": """The merchant name mentioned in the query, if present. For example, "McDonald's". If no merchant is specified, this should be null."""
                },
                "spending_category": {
                    "type": "string",
                    "enum": ['GROCERIES', 'ENTERTAINMENT', 'EATING OUT', 'COFFEE', 'FINANCE', 'BILLS'],
                    "nullable": True,
                    "description": "The spending category mentioned in the query, if present. If no category is mentioned but only a merchant, this must be null."
                },
                "sql_aggregation": {
                    "type": "string",
                    "enum": ["SUM", "AVG", "COUNT", "MAX", "MIN"],
                    "nullable": True,
                    "description": "The SQL aggregation function required to answer the query. Example: 'SUM' for total spend, 'COUNT' for the number of transactions."
                    "If no aggregation function is needed and the user wants to see its transations, this must be null."
                }
            },
            "required": ["start_date", "end_date", "sql_aggregation"]
        }
    }
]

In [ ]:
@timing_decorator
def preprocess_user_query(user_query):

    """
    Uses OpenAI to correct spelling, normalize language, and make the query more structured.
    """

    prompt_preprocess = PromptTemplate.from_template("""
    Given a user query about their transactions, perform these two tasks to preprocess the query for further analysis:
                                                     
        Task 1: correct spelling and grammar while ensuring key financial terms remain intact. If a merchant name is misspelled, replace with the correct spelling.
                Example input: "how muc i spnd at fod sop at Weitros in lastt 3 months?"
                Example output: "How much did I spend at food shop at Waitrose in the last 3 months?"

                Example input: "how much i spnd at Mcdonald two month go?"
                Example output: "How much did I spend at McDonald's two months ago?"
                                                     
        Task 2: if a spending category is mentioned, normalize it to a standard category.
                Use this mapping structure to map user input to **ONLY** these standard categories GROCERIES, ENTERTAINMENT, EATING OUT, COFFEE, FINANCE, BILLS. 
                {CATEGORY_MAPPING}
                
                NOTE that the values of the provided mapping are not exhaustive lists of synonyms for each standard categories.

                Example input:  "How much did I spend at food shop at Waitrose in the last 3 months?"
                Example output: "How much did I spend in GROCERIES at Waitrose in the last 3 months?"
                                                     
                Example input:  "How much did I spend for brunch from 01/12/2024 to 01/01/2025?"
                Example output: "How much did I spend in EATING OUT for brunch from 01/12/2024 to 01/01/2025?"
    
    User query: "{user_query}"
                                                     
    Return only the query after processing the above tasks. Do not include any other text in your answer
    Normalized query:
    """)

    prompt = prompt_preprocess.format( user_query=user_query, CATEGORY_MAPPING = CATEGORY_MAPPING)

    response = client.chat.completions.create(
        model = llm_model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )
    norm_query = response.choices[0].message.content.strip()
    
    return norm_query


def generate_sql_query(start_date, end_date, merchant, spending_category, sql_aggregation):

    """
    Generates an SQL query based on the extracted user's query parameters.
    If a merchant is provided, the query filters by that merchant.
    If a spending category is provided, the query filters by that category.
    If an SQL aggregation function is provided, the query aggregates the amount based on that function.
    If no aggregation function is provided, this means that the user is asking for a set of transactions. Then the query returns the amount, spending category, and merchant.
    """

    if sql_aggregation:
        select_string = f"{sql_aggregation}(amount) AS amount"

    else:
        select_string = "amount, transactions_view.spending_cat, transactions_view.merchant"
    
    base_query = f"""
        WITH transactions_view AS (
        SELECT transaction_id, date, amount, def_merchants.spending_cat, def_merchants.merchant FROM transactions LEFT JOIN def_merchants ON transactions.merchant_id = def_merchants.merchant_id)
        
        SELECT {select_string}
        FROM transactions_view 
        WHERE date BETWEEN ? AND ? 
        """    

    query_params = [start_date, end_date]

    # Add merchant filter if provided
    if merchant:
        base_query += " AND merchant = ?"
        query_params.append(merchant.upper())

    # Add spending category filter if provided
    if spending_category:
        base_query += " AND spending_cat = ?"
        query_params.append(spending_category.upper())
    #print(base_query)
    return base_query, query_params  # Return query string and parameters


def call_database(sql_query, query_params):

    """
    Call the database with the generated SQL query and return the results.
    """

    try:
        # Connect to SQLite database
        conn = sqlite3.connect(db_name)
        cursor = conn.cursor()
        
        # Execute the query
        cursor.execute(sql_query, query_params)

        # Fetch all results
        results = cursor.fetchall()

        # Commit and close connection
        conn.commit()
        conn.close()
        #print(results)
        return results  # Return the fetched query results

    except sqlite3.Error as e:
        print(f"Database error: {e}")
        return None

@timing_decorator
def function_calling_db(user_query):

    """
    Function that:
        1) Preprocess the user query
        2) Extract parameters from the user query
        3) Generate SQL query based on the extracted parameters
        4) Call the database with the generated SQL query

    """
    client = openai.OpenAI()
    log_error = []
    # Get today's date in YYYY-MM-DD format
    today_date = datetime(2025, 3, 4).strftime("%Y-%m-%d")
    day_of_week = datetime(2025, 3, 4).strftime("%A")
    
    # 1) Preprocess the user query
    preprocessed_user_query, preprocess_time = preprocess_user_query(user_query)

    # 2) Extract parameters from the user query
    prompt_enhanced = PromptTemplate.from_template("{preprocessed_user_query} (Today is {today_date}, {day_of_week})")

    prompt = prompt_enhanced.format( preprocessed_user_query = preprocessed_user_query, today_date=today_date, day_of_week=day_of_week)

    # Call OpenAI API with function calling
    response = client.chat.completions.create(
        model=llm_model,
        messages=[{"role": "system", "content": "You extract parameters from a user query"},
                  {"role": "user", "content": prompt}],
        functions = function_calling,
        function_call="auto"
    )
    
    # Check if function calling response is present
    if response.choices[0].message.function_call:
        function_args = json.loads(response.choices[0].message.function_call.arguments)
        #print(function_args)

        # Extract required parameters
        start_date = function_args.get("start_date")
        end_date = function_args.get("end_date")
        merchant = function_args.get("merchant")
        if merchant:
            spending_category = None
            merchant = merchant.upper()
        else:
            spending_category = function_args.get("spending_category")
            if spending_category:
                spending_category = spending_category.upper()
        sql_aggregation = function_args.get("sql_aggregation")

        if spending_category not in list(CATEGORY_MAPPING.keys()) and spending_category is not None:
            log_error.append('the extracted spending_category is not in the predefined list of spending categories')
        if preprocess_time > preprocessing_time_threshold:
            log_error.append(f'Worning: the preprocessing time is above {preprocessing_time_threshold} seconds')
        

        # 3) Call SQL query generator
        sql_query, query_params = generate_sql_query(start_date, end_date, merchant, spending_category, sql_aggregation)

        # 4) Call the database with the generated SQL query
        
        #print(sql_query, query_params)
        db_results = call_database(sql_query, query_params)
        if sql_aggregation:
            df_results = pd.DataFrame(db_results, columns=['amount'])
        else:
            df_results = pd.DataFrame(db_results, columns=['amount', 'category', 'merchant'])
        
        if db_results:
            result = df_results
        else:
            result = None

        return preprocessed_user_query, start_date, end_date, merchant, spending_category, sql_aggregation, result, preprocess_time, log_error

    
    log_error.append("the query does not contain extractable fields")
    preprocessed_user_query, start_date, end_date, merchant, spending_category, sql_aggregation, result, preprocess_time = None, None, None, None, None, None, None, None

    return preprocessed_user_query, start_date, end_date, merchant, spending_category, sql_aggregation, result, preprocess_time, log_error

def return_response(user_query, print_results = False):
    """
    Function that returns the response to the user query in a nicely formatted way.
    """
    
    start_time = time.time()  # Start timing
    
    # Call the main function
    (full_result, function_time) = function_calling_db(user_query)
    preprocessed_user_query, start_date, end_date, merchant, spending_category, sql_aggregation, db_results, preprocess_time, log_error = full_result

    if db_results is not None:
        res = list(db_results.amount.values)
    else:
        res = db_results

    end_time = time.time()  # End timing
    total_execution_time = end_time - start_time  # Calculate execution time

    if total_execution_time > 10:
        log_error.append("Warning: the total execution time is above 10 seconds")
        
    results_data = []
    # Store results
    results_data.append({
        "Original Query": user_query,
        "Cleaned Query": preprocessed_user_query,  # Store cleaned version
        "start_date": start_date,
        "end_date": end_date,
        "merchant": merchant,
        "spending_category": spending_category,
        "SQL_aggregation": sql_aggregation,
        "Response": res,
        "Preprocessing time (s)": preprocess_time,
        "Total Time (s)": total_execution_time,
        "Log errors": log_error
        }
    )
    df_results = pd.DataFrame(results_data)
    
    if print_results:
        if res is not None:
            # Print the results
            print(f"\n\n\nUser Query: {user_query}\n")
            print(f"Preprocessed Query: {preprocessed_user_query}\n")
            print(f"Start Date: {start_date}")
            print(f"End Date: {end_date}")
            print(f"Merchant: {merchant}")
            print(f"Spending Category: {spending_category}")
            print(f"SQL Aggregation: {sql_aggregation}\n")
            print(f"Query Results: {res}\n"),
            print(f"Preprocessing time (s): {preprocess_time}"),
            print(f"Total Execution Time (s): {total_execution_time}")
            print(f"Log errors: {log_error}")
        else:
            print("No results found for the query.")
    
    return df_results

# For parallelization of the queries
def process_query(user_query):
    return return_response(user_query)

# TESTING

### Single query case

In [ ]:
# Write a query here:

user_query = "How much did I spend on groceries between January and June?"
user_query = "how much did I spend on Feb 27th?"

# results are printed here:

res = return_response(user_query, print_results = True)

### Multiple queries

A test set is run in parallel, results are stored in a df (`df_testing_results`). 
A benchmark df (`benchmarks`) is imported and the results from `df_testing_results` are compared for validation.

In [ ]:
# Compute the testing df

user_queries = ["How much did I spend on groceries in the last 4 months?",
                "What was the total spend at McDonald's last week?", 
                "What was the max I have spent on groceries in the last year?",
                "How much did I spend on cinemas in the last year?",
                "wht was my spending on fod in the last 3 months?",                             # spelling mistakes and normalization
                "how much did I invest in Starling since January 1st, 2025?",                   # alternative date formatting
                "how much did I invest since January 1st, 2025?",                               # test that filters only by FINANCE 
                "how many times I went to the cinema in the last 3 months?",                    # test the count
                "What was the total spend at Tesco?",                                           # test that you it works without a start date
                "What was the average spending at Weitrose?",                                   # test the average and the spelling of a brand
                "What was the overall spending at Starling between 01/03/2025 and 11/03/2025?", # testing the ambiguous date format
                "What was the average spending at Tesco between 13/01/2025 and 20/04/2025?",    # testing the date range (DD-MM-AAAA)
                "what are all my transactions at Starling?",                                    # testing that it returns the transactions
                "who is the president of the United States?",                                   # testing that it does not return anything if the query is not related to transactions
                 ] 



# Run queries in parallel
df_testing_results = pd.DataFrame()

with concurrent.futures.ThreadPoolExecutor() as executor:
    results = list(executor.map(process_query, user_queries))

# Concatenate results
#df_testing_results = pd.concat(results, ignore_index=True)

# 🔹 Silence the FutureWarning when using pd.concat
with warnings.catch_warnings():
    warnings.simplefilter(action="ignore", category=FutureWarning)
    df_testing_results = pd.concat(results, ignore_index=True)


# Look at the column PASSED TEST? to see if the test passed or failed. Look at Log errors for more information on errors.
comparison_df = validation(df_testing_results)

# percentage of passed test cases
print(f"""*********\nThe percentage of passed test is:\n\n {comparison_df['PASSED TEST?'].value_counts(normalize=True).iloc[0]*100:.0f}%\n
total execution time is {round(comparison_df['Total Time (s)'].mean(), 2)} seconds on average\n\n""")

comparison_df 


END

Additional notes on project:

## **Fast Inference**

Currently, the response is processed in **about 4 seconds**, well below the **10-second limit**. The **preprocessing step** is the most time-consuming, while the SQL query itself is generated **almost instantly** since it is **statically created** from extracted features.

> **Note:** Using an **LLM to generate the SQL query** would offer more **flexibility** but at the cost of **longer processing times** and potential **inconsistencies** due to hallucinations.

The **database query execution** is **virtually instantaneous** since the system uses a **SQLite database** with only a few columns. However, on a **real database with millions of rows**, inference speed would depend on:  
- **Network latency** (if the database is remote)  
- **Database size**  

### **Optimizing Query Efficiency**

Currently, the SQL query:  
1. **Creates a view** by **joining** the `transactions` and `merchant_id` tables.  
2. **Applies filtering** using the `WHERE` clause.  

However, **this approach is inefficient** when the user requests a summary for a **specific date range**. Since **`JOINs` are generally more expensive than `WHERE`**, a more efficient approach would be:  
- **Filter by date first** in the `transactions` table (assuming it is much larger than `merchant_id`).  
- **Perform the `JOIN` operation** only on the filtered subset.  

This reduces the number of rows being joined, improving performance significantly.

If inference is still a problem, multiple Databases could be used. For instance databases that are specific to a country to reduce the size of the DB itself (if a coutry does not have a TESCO there is no need to have it in the table for that coutry).


## **Scalability**

The process is modular, allowing each step to be individually improved. The AI task is divided into two main stages—preprocessing the user query and feature extraction—to enhance consistency and optimize each task separately. 

For example, the feature extraction step could be executed multiple times in parallel to identify consistently detected features (which should be retained) and discard those that are infrequently identified.

SQL operations are handled independently, ensuring flexibility in database selection. If a NoSQL database (e.g., MongoDB) is used, the AI-extracted features remain valid—it would simply require inserting them into a different precompiled format.

#### **Handling LLM API Limits**
LLM APIs enforce limits on the number of calls per day, typically measured in **Requests Per Minute (RPM)** or **Tokens Per Minute (TPM)**. If these limits are reached, the system should seamlessly switch to backup solutions, such as:

- **Alternative Closed-Source LLMs**
- **Open-Source LLMs**
- **Dedicated Smaller LLMs**: A lightweight, fine-tuned model running on a small GPU cluster could handle specific tasks like grammar correction and feature extraction. Open-source models designed for such tasks would be ideal because:
  - The task is generally simple and does not require complex reasoning.
  - It involves processing a small number of tokens in both input and output.
  
- **Caching Mechanism**: User queries could be temporarily stored for a few days. If an identical query is submitted again, the cached response would be retrieved, bypassing redundant database calls.

A process implemeting cache is diplayed below:


In [ ]:
load_image('processing_with_cache.png', dpi = 300, scale = 1.2)